# Dynamic programming

Dynamic programming is similar to the divide and conquer approach except that the subproblems overlap i.e. the subproblems share the same subsubproblems. This method of solving problems typically is used in optimization problems, particularly optimal control problems, and was originally developed by Richard E. Bellman who is also famous for developing the closely related Bellman equation which is at the core of reinforcement learning {cite:p}`sutton2018reinforcement`.

The key strategy is to store the solutions, in a hashtable for example, to the subproblems and then reuse the solution later if the same subproblem is encountered rather than recomputing the solution {cite:p}`cormen2022introduction`. Computing the $n$-th Fibonacci number is a common example problem that illustrates this.

## Rod cutting

Let's go over the first example of an application of dynamic programming from {cite:p}`cormen2022introduction`.

Suppose we have a rod of length $n \in \mathbb{Z^+}$ and we can sell the rod for a price $p_n$ which is the cash we get for selling a rod of length $n$. We can *also* cut the rod into $k$ parts of integer length $c_i$ such that (obviously)

$$
\begin{align*}
n = c_1 + c_2 + \dots + c_k \quad \text{where $c_i \in \mathbb{Z^+}$}
\end{align*}
$$

and sell each part for the revenue $r_n = p_{c_1} + p_{c_2} + \dots + p_{c_k}$ where again $p_{c_i}$ is the cash we get for selling a rod of integer length $c_i$. The caveat here is that there maybe certain ways to cut the rod such that we actually get more profit by selling the individually cut pieces instead of selling the uncut rod. 
Clearly this depends on the pricing for each length of rod.  Can we devise an algorithm that, given the length $n$ of the rod along with the prices $p_1, p_2,\dots, p_n$ for selling a rod of length $1, 2, \dots, n$ respectively, finds an (as there could be more than one I think) optimal way of cutting the rod such that we net the highest revenue?

### Mathematical reasoning for solution

Suppose that an optimal way of cutting the rod (optimal integer decomposition of $n$) cuts the rod into $k$ pieces for some integer $1 \leq k \leq n$. We denote this optimal decomposition as

$$
n_{opt} = c_1 + c_2 + \dots + c_k
$$

where $c_i$ is the length of the $i$-th segment of the rod. The corresponding optimal revenue is thus

$$
r_{n_{opt}} = p_{c_1} + p_{c_2} + \dots + p_{c_k}.
$$

In general we can write the optimal revenue $r_{n_{opt}}$ in terms of optimal revenues for rods of shorter length

$$
r_{n_{opt}} = \max(p_n, r_{1_{opt}} + r_{(n-1)_{opt}}, r_{2_{opt}} + r_{(n-2)_{opt}}, \dots, r_{(n-1)_{opt}} + r_{1_{opt}} )
$$

where $p_n$ is the revenue gained from selling the uncut rod of length $n$. The other $n-1$ arguments to $\max$ correspond to the revenue gained from cutting the rod into two pieces of size $i$ and $n-i$ respectively $\forall i \in \{1,2,\dots,n-1\}$ and then optimally cutting up the two resulting pieces of the rod further.

If you think about it it makes sense that the optimal solution for this problem can be achieved by combining optimal solutions of smaller problems. For example when you start with the uncut rod the two options are either 
1) the uncut rod is already optimal to sell 

$\Large{\textbf{or}}$

2) there is a way to cut it to get more revenue in which case you know you have to cut it into ***at least*** two pieces. 

After that point you have two new rods which you need to find the optimal cutting/decomposition of and you simply repeat the previous step again for each rod. The problem here is you need to cut the rod into two pieces in ***every*** possible way since you don't know ahead of time which way of cutting the rod into two pieces (i.e. which value of $i$) is optimal hence all the $n-1$ arguments to $\max$. 

```{note}
:class: dropdown
When a problem can be solved like this by breaking it onto smaller parts, solving those smaller parts independently, and then combining those solutions to those smaller parts we say the problem exhibits **optimal sub-structure**.
```

In [30]:
from random import randint

"""
p: 

n: 
"""

def naive_rod_cutting(p, n):
    if n == 0:
       return 0
    
    max_rev = float('-inf') # initially set revenue to lowest possible number
    
    for i in range(1, n+1):
        max_rev = max(max_rev, p[i] + naive_rod_cutting(p, n-i)) 
    
    return max_rev


#p = { 1:1, 2:5, 3:8, 4:9, 5:10, 6:17, 7:17, 8:20, 9:24, 10:30,  }
p = {  i:i*randint(1, i)  for i in range(1, 51)  }
print(f"p = {p}")
print("Max revenue is:", naive_rod_cutting(p, 40))

p = {1: 1, 2: 4, 3: 3, 4: 12, 5: 10, 6: 12, 7: 7, 8: 40, 9: 9, 10: 40, 11: 55, 12: 144, 13: 143, 14: 14, 15: 135, 16: 80, 17: 51, 18: 144, 19: 76, 20: 240, 21: 357, 22: 440, 23: 437, 24: 48, 25: 250, 26: 598, 27: 594, 28: 84, 29: 754, 30: 90, 31: 496, 32: 32, 33: 99, 34: 170, 35: 735, 36: 108, 37: 1184, 38: 114, 39: 858, 40: 680, 41: 943, 42: 1638, 43: 1677, 44: 1100, 45: 1890, 46: 1840, 47: 517, 48: 864, 49: 539, 50: 1400}


KeyboardInterrupt: 

In [ ]:
def naive_rod_cutting(p, n, memo):
    if n == 0:
       return 0
    
    max_rev = float('-inf') # initially set revenue to lowest possible number
    
    for i in range(1, n+1):
        max_rev = max(max_rev, p[i] + naive_rod_cutting(p, n-i)) 
    
    return max_rev

## $n$-th Fibonacci number

In [ ]:
def nth_fibonacci(n):
    pass